In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `

In [3]:
adata = sc.read("../data/model_set.h5ad", backed='r')

In [4]:
# Set example parameters
n_indices = adata.n_obs
n_classes = adata.obs.cls_label.nunique()

In [5]:
# Determine optimal batch size for data
min_cells_per_batch = 12
max_cells_per_batch = 24
#min_classes_per_batch = int(batch_size / max_cells_per_batch)
#max_classes_per_batch = int(batch_size / min_cells_per_batch)
min_classes_per_batch = 10
max_classes_per_batch = 20

In [6]:
# Go for bigger batch size
batch_size = max_cells_per_batch * max_classes_per_batch

In [7]:
indices = adata.obs_names.values

In [8]:
cls_labels = adata.obs.cls_label.factorize()[0]

In [9]:
cpp = pd.Series(cls_labels).value_counts()

In [18]:
import os

os.chdir('../')

In [25]:
from src.data._contrastive_sampler import ContrastiveBatchSampler

cs = ContrastiveBatchSampler(indices, cls_labels)
bsr = cs.sample(return_details=True)

In [34]:
batched_adata = adata[bsr[ContrastiveBatchSampler.IDX_KEY]]

In [39]:
adata.obs.cls_label.value_counts()

cls_label
neg;TFAM      6589
neg;GFM1      3088
neg;MRPL36    2936
neg;TARDBP    2518
neg;MRPL35    2429
              ... 
neg;CTPS1       62
neg;NAGLU       62
neg;RNF14       59
neg;MT2A        58
neg;ZNF100      56
Name: count, Length: 713, dtype: int64

In [36]:
batched_adata.obs[:batch_size].cls_label.value_counts()

cls_label
neg;PHB2       24
neg;SNRNP40    24
neg;DNMT1      24
neg;USP14      24
neg;RHOQ       24
               ..
neg;IPO9        0
neg;ISCA2       0
neg;JAZF1       0
neg;JMJD6       0
neg;ING3        0
Name: count, Length: 713, dtype: int64

In [37]:
batched_adata.obs[batch_size:(batch_size*2)].cls_label.value_counts()

cls_label
neg;RNASEH2C    24
neg;DNAJC8      24
neg;MRPL34      24
neg;RPL8        24
neg;ISCA2       24
                ..
neg;INTS9        0
neg;INTS13       0
neg;INTS14       0
neg;IPO9         0
neg;IK           0
Name: count, Length: 713, dtype: int64